In [86]:
import cv2
import os
import skimage
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import math
import tqdm
from skimage.feature import local_binary_pattern
from timeit import default_timer as timer
import pytesseract
import string
#pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ialak\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

In [2]:
def get_dict_from_data(path, extension=".jpg"):
    dictionary = dict()
    for file in sorted(os.listdir(path)):
        if file.endswith(extension):
            key = os.path.splitext(file)[0].split('_').pop()
            if extension == ".txt":
                f = open(os.path.join(path, file), "r",encoding = "ISO-8859-1").read()
                line = f.strip().strip('()')
                parts = line.split(', ')
                author = parts[0].strip("''")
                dictionary[key] = author
            else:
                dictionary[key] = cv2.imread(os.path.join(path, file))
    return dictionary

In [3]:
BBDD_PATH = './BBDD/'
QSD1_W3_PATH = './qsd1_w3/'
QSD2_W3_PATH = './qsd2_w3/'

bbdd = get_dict_from_data(BBDD_PATH)
qsd1_w3 = get_dict_from_data(QSD1_W3_PATH)
qsd2_w3 = get_dict_from_data(QSD2_W3_PATH)

In [4]:
qsd1_w3_text_labels = get_dict_from_data(QSD1_W3_PATH, extension=".txt")

### We create a dictionary with all the author names of the BBDD

In [5]:
def get_authors(path):
    authors = ["unknown"]

    for file in sorted(os.listdir(path)):
        if file.endswith(".txt"):
            f = open(os.path.join(path, file), "r", encoding = "ISO-8859-1").read()
            line = f.strip().strip('()')
            parts = line.split(', ')
            author = parts[0].strip("''")
            if len(author) == 0:
                continue
            author = author.lower()
            
            
            if author not in authors:
                authors.append(author)
    
    return authors

In [6]:
authors = get_authors(BBDD_PATH)

In [7]:
sorted(authors)

['agusti puig',
 'albert rafols-casamada',
 'alfons borrell',
 'alfonso alzamora',
 'alfred figueras',
 'amelia riera',
 'anders svarstad',
 'andre lhote',
 'anna-eva bergman',
 'antoni clave',
 'antoni llena',
 'antoni tapies',
 'arranz-bravo',
 'assumpcio mateu',
 'benet rossell',
 'bruno olle',
 'corneille',
 'daniel cuervo',
 'daniel enkaoua',
 'diego rivera',
 'edvard munch',
 'elias fiigenschou',
 'enric ansesa',
 'enric barro',
 'enrique barro',
 'fjosfrieri',
 'francesc artigau',
 'francesc ruestes',
 'francesca llopis',
 'frederic amat',
 'gerard fernandez-rico',
 'gerard sala',
 'gino rubert',
 'gonzalo goytisolo',
 'gregori iglesias',
 'hugo demarco',
 'jais nielsen',
 'jan moritz kaland',
 'jens ferdinand willumsen',
 'jesus rafael soto',
 'joan hernandez pijuan',
 'joan josep tharrats',
 'joan longas',
 'joan pere viladecans',
 'joan ponc',
 'joaquim chancho',
 'jordi alcaraz',
 'jordi bernado',
 'jordi fulla',
 'jordi isern',
 'jordi prat',
 'jose luis pascual',
 'jose m.

In [8]:
author_names = get_dict_from_data(QSD1_W3_PATH, extension=".txt")

## TASK 1

### FIlter noise with linear or non-linear filters

In [9]:
# Noise estimation using Change Detection

def Change_Detection(img):
    
    h, w, c = img.shape
    
    pixel_avg_c0 = np.sum(img[:, :, 0]) / (h*w)
    pixel_avg_c1 = np.sum(img[:, :, 1]) / (h*w)
    pixel_avg_c2 = np.sum(img[:, :, 2]) / (h*w)
    
    change_c0 = pixel_avg_c0 - img[:, :, 0]
    change_c1 = pixel_avg_c1 - img[:, :, 1]
    change_c2 = pixel_avg_c2 - img[:, :, 2]
    
    change = np.zeros(img.shape, dtype = "uint8")
    change[:, :, 0] = change_c0
    change[:, :, 1] = change_c1
    change[:, :, 2] = change_c2
    
    scaled_change_c0 = ((change_c0 - change_c0.min()) * (1/(change_c0.max() - change_c0.min()) * 255)).astype('uint8')
    scaled_change_c1 = ((change_c1 - change_c1.min()) * (1/(change_c1.max() - change_c1.min()) * 255)).astype('uint8')
    scaled_change_c2 = ((change_c2 - change_c2.min()) * (1/(change_c2.max() - change_c2.min()) * 255)).astype('uint8')
    
    #scaled_change = np.zeros(img.shape, dtype = "uint8")
    #scaled_change[:, :, 0] = scaled_change_c0
    #scaled_change[:, :, 1] = scaled_change_c1
    #scaled_change[:, :, 2] = scaled_change_c2
    
    original_minus_change = img - change
    #original_minus_change = ((original_minus_scaled_change - original_minus_scaled_change.min()) * (1/(original_minus_scaled_change.max() - original_minus_scaled_change.min()) * 255)).astype('uint8')
    
    
    fig, ax = plt.subplots(nrows=1, ncols=6, figsize = (10, 10))
    
    ax[0].imshow(img)
    ax[1].imshow(scaled_change_c0, cmap = "gray")
    ax[2].imshow(scaled_change_c1, cmap = "gray")
    ax[3].imshow(scaled_change_c2, cmap = "gray")
    ax[4].imshow(change)
    ax[5].imshow(original_minus_change)


In [10]:
def get_medians(qs):
    
    result = dict()
    
    for name, img in qs.items():
        median_img = cv2.medianBlur(img, 3)
        result[name] = median_img
        
    return result

In [11]:
def get_gaussians(qs):

    result = dict()

    for name, img in qs.items():
        gauss_img = cv2.GaussianBlur(img, (3,3),0)
        result[name] = gauss_img

    return result

In [12]:
def get_average(qs):

    result = dict()

    for name, img in qs.items():
        kernel = np.ones((7,7),np.float32)/49
        dst = cv2.filter2D(img,-1,kernel)
        result[name] = dst

    return result

In [13]:
def PSNR(original, noisy): 
    mse = np.mean((original - noisy) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse)) 
    return psnr

In [14]:
def get_all_PSNR(qs_orig,qs_noise):
    total =[]
    for name, img in qs_orig.items():
        valor = PSNR(img, qs_noise[name])
        total.append(valor)
    return np.mean(total)

In [15]:
qsd1_w3_median = get_medians(qsd1_w3)
qsd1_w3_gaussian = get_gaussians(qsd1_w3)
qsd1_w3_avg = get_average(qsd1_w3)

PSNR_median = get_all_PSNR(qsd1_w3, qsd1_w3_median)
PSNR_gaussian = get_all_PSNR(qsd1_w3, qsd1_w3_gaussian)
PSNR_avg = get_all_PSNR(qsd1_w3, qsd1_w3_avg)

print(f"Mean PSNR with median filter: {PSNR_median}")
print(f"Mean PSNR with Gaussian filter: {PSNR_gaussian}")
print(f"Mean PSNR with average filter: {PSNR_avg}")

Mean PSNR with median filter: 35.25212159675256
Mean PSNR with Gaussian filter: 34.298950335284225
Mean PSNR with average filter: 31.36960674226446


## TASK 2

### Detect box with overlapping text on denoised images. The apply OCR to get the text

In [16]:
def get_name_boxs(img):
    
    h_i, w_i, c = img.shape
    
    top_limit = int(round(h_i/6))
    bottom_limit = int(h_i - top_limit)
    
    left_area = int(round(w_i/3))
    right_area = int(left_area*2)
    
    area_pixel= int(h_i*w_i)
    
    # BGR to Lab conversion, and take L channel only
    image = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    imageL = image[:, :, 0]
    
    # Black hat and top hat separately to the L channel image
    kernel = np.ones((2, 10),np.uint8)
    thL = cv2.morphologyEx(imageL, cv2.MORPH_TOPHAT, kernel, iterations = 5)
    bhL = cv2.morphologyEx(imageL, cv2.MORPH_BLACKHAT, kernel, iterations = 5)
    
    # Take the highest value pixels, mostly letters
    fthL_th = thL.max() - 50
    fthL = (thL[:, :] > fthL_th) * 255
    fthL = fthL.astype('uint8')    
        
    fbhL_th = bhL.max() - 50
    fbhL = (bhL[:, :] > fbhL_th) * 255
    fbhL = fbhL.astype('uint8')
    
    # OR between top hat and black hat most important pixels
    mix = fbhL + fthL
    mix = mix>1
    mix= mix.astype('uint8')
    
    # Closing
    kernel = np.ones((20, 30),np.uint8)
    closing = cv2.morphologyEx(mix, cv2.MORPH_CLOSE, kernel)
    
    
    # Connected Components
    analysis = cv2.connectedComponentsWithStats(closing,4, cv2.CV_32S) 
    (totalLabels, label_ids, values, centroid) = analysis 

    # Initialize a new image to store  
    # all the output components 
    output = np.zeros(imageL.shape, dtype="uint8") 

    # Loop through each component 
    mask = np.zeros([h_i,w_i], dtype=np.uint8)
    cont=0
    x1_prev= -10000
    for i in range(1, totalLabels): 
        
        # Area of the component 
        area = values[i, cv2.CC_STAT_AREA]  
        
        (X, Y) = centroid[i]
        
        if (area >round(area_pixel*0.0009)) and ((Y<top_limit)or(Y>bottom_limit))and ((X>left_area)and(X<right_area)): 
            x1 = values[i, cv2.CC_STAT_LEFT] 
            y1 = values[i, cv2.CC_STAT_TOP] 
            w = values[i, cv2.CC_STAT_WIDTH] 
            h = values[i, cv2.CC_STAT_HEIGHT]
            
            pt1 = (x1, y1) 
            pt2 = (x1+ w, y1+ h)
            if (int(x1 - x1_prev) < int(round(w_i/4)) and (int(x1 - x1_prev) > int(round(-w_i/4)))):
                roi_corners = np.array([(x1_prev,y1_prev),(x1+w,y1), pt2,(x1_prev,y1+h)], dtype=np.int32)
            else:
                roi_corners = np.array([pt1,(x1+w,y1), pt2,(x1,y1+h)], dtype=np.int32)
                
            cv2.fillPoly(mask, [roi_corners], 255)
            output = cv2.bitwise_or(output, mask)
            cont = cont+1
            x1_prev= x1
            y1_prev = y1                           
                        
    if cont == 0:
        output = fthL
        
        
    # Closing
    kernel2 = np.ones((int(h_i*0.001), int(w_i*0.03)),np.uint8)
    closing2 = cv2.morphologyEx(output, cv2.MORPH_CLOSE, kernel2)
    
    # Dilation
    kernel3 = np.ones((int(h_i*0.03), int(w_i*0.003)),np.uint8)
    dilate = cv2.dilate(closing2,kernel3)

    #print(thL.max())
    #print(np.count_nonzero((fthL < 350)&(fthL > 0)))
    
    # For visualization:
    """
    plt.axis("off")
    fig, ax = plt.subplots(nrows=3, ncols=3, figsize = (10, 10))
    
    ax[0, 0].imshow(dilate, cmap = "gray")
    ax[0, 1].imshow(closing, cmap = "gray")
    ax[0, 2].imshow(closing2, cmap = "gray")
    
    ax[1, 0].imshow(thL, cmap = "gray")
    ax[1, 1].imshow(fthL, cmap = "gray")        
    ax[1, 2].imshow(mix, cmap = "gray")
    
    ax[2, 0].imshow(bhL, cmap = "gray")
    ax[2, 1].imshow(fbhL, cmap = "gray")
    ax[2, 2].imshow(output, cmap = "gray")
    """
    
    return dilate

In [17]:
def get_tl_and_br(mask):
    
    xl = 100000
    xr = -1
    yu = 100000
    yd = -1
    
    for y in range(mask.shape[0]):
        for x in range(mask.shape[1]):
            if mask[y, x] == 255:
                if x < xl: xl = x
                if x > xr: xr = x
                if y < yu: yu = y
                if y > yd: yd = y
    
    return (xl, yu), (xr, yd)

In [18]:
def get_pred_cords(masks_dict):

    result = []
    for name, mask in masks_dict.items():
        tl, br = get_tl_and_br(mask)
        result.append([tl[0], tl[1], br[0], br[1]])
        
    return result

In [19]:
def get_pred_masks(qs):
    
    masks_dict = dict()
    
    for name, img in qs.items():
        masks_dict[name] = get_name_boxs(img)
    
    return masks_dict

In [20]:
def get_rectangular_predicted_masks(qs):
    masks_dict = dict()
    
    
    for name, img in qs.items():
        not_rect_mask = get_name_boxs(img)
        
        tl, br = get_tl_and_br(not_rect_mask)
        tr = (br[0], tl[1])
        bl = (tl[0], br[1])
        
        #print(f"tl: {tl}, tr:{tr}, bl: {bl}, br: {br}")
        
        rect_mask = np.zeros((img.shape[0], img.shape[1]), dtype = np.uint8)
        roi_corners = np.array([tl, tr, br, bl], dtype = np.int32)
        cv2.fillPoly(rect_mask, [roi_corners], 255)
        
        masks_dict[name] = rect_mask
        
    return masks_dict

In [21]:
def apply_mask(querys, masks):
    
    masked_qs = dict()
    
    for name, img in querys.items():
        img_mask = masks[name]
        
        masked_image = cv2.bitwise_and(img, img, mask = img_mask.astype(np.uint8))
        masked_qs[name] = masked_image

    return masked_qs

In [22]:
tb_masks_qsd1_w3 = get_rectangular_predicted_masks(qsd1_w3_median)

In [23]:
qsd1_w3_masked = apply_mask(qsd1_w3_median, tb_masks_qsd1_w3)

## Text Distance Metrics

https://activewizards.com/content/blog/Comparison-of-the-Text-Distance-Metrics/text-distance-infographics-table05-(1).png

There exist a lot of different distance metrics for text:
- Edit Based Similarities
- Token Based Similarities
- Sequence Based
- Phonetic
- Simple

In [24]:
# Edit Based Similarities: Levenshtein Algorithm

# !python3 -m pip install nltk

import nltk
from sklearn.metrics import jaccard_score

def levenshtein(actual, predicted):
    return nltk.edit_distance(actual, predicted)

def jaro_winkler(actual, predicted):
    return nltk.distance.jaro_similarity(actual, predicted)

In [25]:
import re

def text_distance(actual, predicted, method):
    
    # Remove non-ascii text
    removed_actual = re.sub(r'[^a-zA-Z\s]', '', actual)
    removed_actual = removed_actual.strip()
    
    # Remove non-ascii text
    predicted = re.sub(r'[^a-zA-Z\s]', '', predicted)

    # Remove new lines \n read by OCR
    predicted = predicted.strip()

    #print("Actual: '{}', Predicted: '{}'".format(actual, predicted))

    # Get distance
    if method == "levenshtein":
        return levenshtein(removed_actual, predicted)
    elif method == "jaro_winkler":
        return jaro_winkler(removed_actual, predicted)
    elif method == "jaccard":
        return jaccard_score(removed_actual, predicted)
    else:
        raise Exception("{} method not found".format(method))

In [26]:
def get_closest(authors, predicted, method = "levenshtein"):
    
    distances = []
    
    pred = re.sub(r'[^a-zA-Z\s]', '', predicted)
    # Remove new lines \n read by OCR
    pred = pred.strip()
    pred = pred.lower()
    
    for author in authors:
        if method == "levenshtein": distances.append(levenshtein(author, pred))
        elif method == "jaccard": distances.append(jaccard_score(author, pred))
        elif method == "jaro_winkler": distances.append(jaro_winkler_similarity(author, pred))
        
        
    index = distances.index(min(distances))
    
    return authors[index], min(distances)     

In [27]:
def get_distances(qs_masked, tb_masks, labels, method = "levenshtein"):
    
    distance_normal = 0
    distance_otsu = 0
    distance_laplacian = 0
    
    for name, img_masked in qs_masked.items():
        gt = labels[name]
        
        tl, br = get_tl_and_br(tb_masks[name])
        tlx, tly = tl
        brx, bry = br
        margin = 0
        cropped_img = qs_masked[name][tly - margin : bry, tlx - margin : brx]
        ret2, th2 = cv2.threshold(cropped_img[:, :, 0], 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        laplacian = cv2.Laplacian(qs_masked[name], cv2.CV_8U)
        laplacian = (laplacian > 20).astype(np.uint8) * 255
        
        extractedInformation_normal = pytesseract.image_to_string(cropped_img)
        extractedInformation_otsu = pytesseract.image_to_string(th2)
        extractedInformation_laplacian = pytesseract.image_to_string(laplacian)
        
        normal_split = extractedInformation_normal.split('\n')[0]
        otsu_split = extractedInformation_otsu.split('\n')[0]
        laplacian_split = extractedInformation_laplacian.split('\n')[0]
        
        closest_normal = get_closest(authors, normal_split, method)[0]
        closest_otsu = get_closest(authors, otsu_split, method)[0]
        closest_laplacian = get_closest(authors, laplacian_split, method)[0]

        #print(f"Normal reading: {extractedInformation_normal}")
        #print(f"Laplacian reading: {extractedInformation_otsu}")
        
        # text distance 
        dist_normal = text_distance(gt.lower(), closest_normal, method=method)
        distance_normal +=  dist_normal
        
        dist_otsu = text_distance(gt.lower(), closest_otsu, method=method)
        distance_otsu += dist_otsu
        
        dist_laplacian = text_distance(gt.lower(), closest_laplacian, method=method)
        distance_laplacian += dist_laplacian
        
        print(f"Image: {name}, normal_distance: {dist_normal}, otsu distance: {dist_otsu}, laplacian distance: {dist_laplacian}")
        
    distance_normal /= len(qs_masked)
    distance_otsu /= len(qs_masked)
    distance_laplacian /= len(qs_masked)
    
    return distance_normal, distance_otsu, distance_laplacian
    

In [28]:
mean_normal_distance, mean_otsu_distance, mean_laplacian_distance = get_distances(qsd1_w3_masked, tb_masks_qsd1_w3, qsd1_w3_text_labels)

Image: 00000, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00001, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00002, normal_distance: 0, otsu distance: 0, laplacian distance: 7
Image: 00003, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00004, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00005, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00006, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00007, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00008, normal_distance: 18, otsu distance: 18, laplacian distance: 18
Image: 00009, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00010, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00011, normal_distance: 0, otsu distance: 0, laplacian distance: 9
Image: 00012, normal_distance: 0, otsu distance: 0, laplacian distance: 0
Image: 00013, normal_distance: 0, o

In [29]:
mean_normal_distance, mean_otsu_distance, mean_laplacian_distance

(1.7, 1.6333333333333333, 4.733333333333333)

## TASK 3

### Metrics for histogram comparison

In [31]:
def Euclidean(h1, h2):
    return np.linalg.norm(h1 - h2)

In [32]:
def L1_distance(h1, h2):
    result = np.subtract(h1, h2)
    result = np.absolute(result)
    return np.sum(result)

In [33]:
def XSquaredDistance(h1, h2):
    result = ((h1 - h2)**2 / (h1 + h2))
    result = np.nan_to_num(result, nan = 0.0)
    return np.sum(result)

In [34]:
def HistogramIntersection(h1, h2):
    return np.sum(np.minimum(h1, h2))

In [35]:
def HellingerKernel(h1, h2):
    x = np.multiply(h1, h2)
    x = np.sqrt(x)
    return np.sum(x)

In [36]:
def compare(query, database, method):
    result = dict()
    for k, v in database.items():
        if method == "Euclidean": result[k] = Euclidean(v, query)
        elif method == "L1": result[k] = L1_distance(v, query)
        elif method == "X_Squared": result[k] = XSquaredDistance(v, query)
        elif method == "Histogram_Intersection": result[k] = HistogramIntersection(v, query)
        elif method == "Hellinger_Kernel": result[k] = HellingerKernel(v, query)
        
    return result

In [37]:
def k_neighbours(dictionary, k=10, rev=False):
    result_dict = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=rev))
    return [int(keys) for keys,v in result_dict.items()][:k]

In [38]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [39]:
def calculateMAPK(histograms, histograms_bbdd, ground_truth = None, topK=10):
    apk_list_Euclidean = []
    apk_list_X_Squared = []
    apk_list_L1 = []
    apk_list_Histogram_Intersection = []
    apk_list_Hellinger_Kernel = []
               
    for k, v in histograms.items():
        apk_list_Euclidean += [k_neighbours(compare(v, histograms_bbdd, "Euclidean"), topK, rev = False)]
        apk_list_X_Squared += [k_neighbours(compare(v, histograms_bbdd, "X_Squared"), topK, rev = False)]
        apk_list_L1 += [k_neighbours(compare(v, histograms_bbdd, "L1"), topK, rev = False)]
        apk_list_Histogram_Intersection += [k_neighbours(compare(v, histograms_bbdd, "Histogram_Intersection"), topK, rev = True)]
        apk_list_Hellinger_Kernel += [k_neighbours(compare(v, histograms_bbdd, "Hellinger_Kernel"), topK, rev = True)]
      
    if ground_truth:

        mapk_Euclidean = mapk(actual=ground_truth, predicted=apk_list_Euclidean, k=topK)
        mapk_X_Squared = mapk(actual=ground_truth, predicted=apk_list_X_Squared, k=topK)
        mapk_L1 = mapk(actual=ground_truth, predicted=apk_list_L1, k=topK)
        mapk_Histogram_Intersection = mapk(actual=ground_truth, predicted=apk_list_Histogram_Intersection, k=topK)
        mapk_Hellinger_Kernel = mapk(actual=ground_truth, predicted=apk_list_Hellinger_Kernel, k=topK)
        
        mapk_list = [mapk_Euclidean, mapk_X_Squared, mapk_L1, mapk_Histogram_Intersection, mapk_Hellinger_Kernel]
        
        print(f"Euclidean: {mapk_Euclidean}")
        print(f"X_Squared: {mapk_X_Squared}")
        print(f"L1: {mapk_L1}")
        print(f"Histogram_Intersection: {mapk_Histogram_Intersection}")
        print(f"Hellinger_Kernel: {mapk_Hellinger_Kernel}")
        
        return max(mapk_list)
    else:
        
        return apk_list

In [40]:
def calculateMAPK_w1(histograms, histograms_bbdd, method, ground_truth = None, topK=10, rev = False):
    apk_list = []
              
    for k, v in histograms.items():
        apk_list += [k_neighbours(compare(v, histograms_bbdd, method), topK, rev = rev)]
      
    if ground_truth:
        #print(ground_truth)
        #print(apk_list)
        return mapk(actual=ground_truth, predicted=apk_list, k=topK)
    else:
        return apk_list

In [41]:
# TEACHER'S CODE

def add_list_level(input_list):
    out = []
    for ll in input_list:
        tmp = []
        for q in ll:
            tmp.append([q])
        out.append(tmp)
    return (out)

list_depth = lambda L: (isinstance(L, list) or isinstance(L, tuple)) and max(map(list_depth, L))+1

def compute_mapk(gt,hypo,k_val):

    hypo = list(hypo)
    if list_depth(hypo) == 2:
        hypo = add_list_level(hypo.copy())

    apk_list = []
    for ii,query in enumerate(gt):
        for jj,sq in enumerate(query):
            apk_val = 0.0
            if len(hypo[ii]) > jj:
                apk_val = apk([sq],hypo[ii][jj], k_val)
            apk_list.append(apk_val)
            
    return np.mean(apk_list)

In [42]:
def InvertMasks(mask_dict_orig):
    masks_dict_invert = dict()
    for name, img in mask_dict_orig.items():
        masks_dict_invert[name] = (255-mask_dict_orig[name])
    
    return masks_dict_invert

In [47]:
tb_masks_inverted_qsd1_w3 = InvertMasks(tb_masks_qsd1_w3)

### LBP Descriptors

In [26]:
#lbp = local_binary_pattern(image, n_points, radius, METHOD)


# with the radius we choose how far from the central pixel
# we want to do the lbp

# for a matrix like this:

#   3 3 3 3 3 3 3
#   3 2 2 2 2 2 3
#   3 2 1 1 1 2 3
#   3 2 1 0 1 2 3
#   3 2 1 1 1 2 3
#   3 2 2 2 2 2 3
#   3 3 3 3 3 3 3

# 0 is the central point, and when choosing radius = 1
# we get to analyze the pixels at radius 1, the pixels that
# in this matrix have value 1. When we choose radius = 2
# we get to analyze the pixels which have value = 2 at this
# matrix, and so on. And that's way we have n_point = 8*radius,
# because there are 8*1 pixels with value =1, 8*2 pixels with value
# =2, and 8*3 pixels with value = 3

# radius = 2
# n_points = 8 * radius
# lbp = local_binary_pattern(gray_img, n_points, radius)

In [43]:
def histogram_by_block(img, n_blocks_x, n_blocks_y, bins = 8, dim = "1", mask = None, radius = 1):
    
    M = math.ceil(img.shape[0]/n_blocks_x)
    N = math.ceil(img.shape[1]/n_blocks_y)
    tiles = [img[x:x+M, y:y+N] for x in range(0, img.shape[0], M) for y in range(0, img.shape[1], N)]
    
    if mask is not None: 
        mask_tiles = [mask[x:x+M,y:y+N] for x in range(0, img.shape[0],M) for y in range(0, img.shape[1], N)]
       
    histogram = np.array([])
    
    for i in range(len(tiles)):
        
        #ESTE NO CREO QUE LO VAYAMOS HA USAR
        #if dim == "3":
        #    if mask is not None: tile_hist = cv2.calcHist([tiles[i], tiles[i], tiles[i]], [0, 1, 2], mask_tiles[i], [bins, bins, bins], [0, 256, 0, 256, 0, 256])
        #    else: tile_hist = cv2.calcHist([tiles[i]], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
        #        
        #    if np.sum(tile_hist) != 0:
        #        tile_hist /= np.sum(tile_hist)
            
         
        #ESTE NO CREO QUE LO VAYAMOS HA USAR
        #elif dim == "2d1d":           
        #    if mask is not None: tile_hist = get_2d1d_hist(tiles[i], bins, mask = mask_tiles[i])
        #    else: tile_hist = get_2d1d_hist(tiles[i], bins, mask = None)
           
        if dim == "1" and tiles[i].ndim == 3:
            if mask is not None: 
                tile_hist0 = cv2.calcHist([tiles[i]], [0], mask_tiles[i], [bins], [0, radius*8-1])
                tile_hist1 = cv2.calcHist([tiles[i]], [1], mask_tiles[i], [bins], [0, radius*8-1])
                tile_hist2 = cv2.calcHist([tiles[i]], [2], mask_tiles[i], [bins], [0, radius*8-1])
            else: 
                tile_hist0 = cv2.calcHist([tiles[i]], [0], None, [bins], [0, radius*8-1])
                tile_hist1 = cv2.calcHist([tiles[i]], [1], None, [bins], [0, radius*8-1])
                tile_hist2 = cv2.calcHist([tiles[i]], [2], None, [bins], [0, radius*8-1])
                
            if np.sum(tile_hist0) != 0: tile_hist0 /= np.sum(tile_hist0)
            if np.sum(tile_hist1) != 0: tile_hist1 /= np.sum(tile_hist1)
            if np.sum(tile_hist2) != 0: tile_hist2 /= np.sum(tile_hist2)
            
            tile_hist = np.concatenate((tile_hist0, tile_hist1, tile_hist2))
            
        elif dim == "1" and tiles[i].ndim != 3:
            
            h, w = tiles[i].shape[0], tiles[i].shape[1]
            
            tile = np.zeros((h, w, 3), dtype = np.uint8)
            tile[:, :, 0] = tiles[i]
            
            if mask is not None: 
                tile_hist = cv2.calcHist([tile], [0], mask_tiles[i], [bins], [0, radius*8-1])
            else: 
                tile_hist = cv2.calcHist([tile], [0], None, [bins], [0, radius*8-1])
            
            if np.sum(tile_hist) != 0: tile_hist /= np.sum(tile_hist)
        
        histogram = np.concatenate((histogram, tile_hist.flatten()))

    return histogram

In [44]:
def get_full_hist_lbp(img, levels = 3, bins = 8, dim = "1", color = "RGB", mask = None, radius = 1, multiscale = False):
    
    h, w, c = img.shape
    
    if color == "Lab":
        image = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)     
    elif color == "Gray":
        image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else: 
        image = img.copy()
        
    if multiscale:

        full_hist = np.array([])

        for rad in range (1, radius+1):
            if color == "Gray": 
                lbp = local_binary_pattern(image, radius*8, rad)
            else:
                lbp0 = local_binary_pattern(image[:, :, 0], rad*8, rad) # n_points = radius*8
                lbp1 = local_binary_pattern(image[:, :, 1], rad*8, rad)
                lbp2 = local_binary_pattern(image[:, :, 2], rad*8, rad)

                lbp = np.zeros((h, w, c), dtype = np.uint8)
                lbp[:, :, 0] = lbp0
                lbp[:, :, 1] = lbp1
                lbp[:, :, 2] = lbp2

            levels_histogram = np.array([])

            for i in range(levels):
                block_hist = histogram_by_block(lbp, 2**i, 2**i, bins = bins, dim = dim, mask = mask, radius = rad) 
                levels_histogram = np.concatenate((levels_histogram, block_hist))

            full_hist = np.concatenate((full_hist, levels_histogram))

        return full_hist

    else: 
        
        if color == "Gray": 
            lbp = local_binary_pattern(image, radius*8, radius)
        else:
            lbp0 = local_binary_pattern(image[:, :, 0], radius*8, radius)
            lbp1 = local_binary_pattern(image[:, :, 1], radius*8, radius)
            lbp2 = local_binary_pattern(image[:, :, 2], radius*8, radius)

            lbp = np.zeros((h, w, c), dtype = np.uint8)
            lbp[:, :, 0] = lbp0
            lbp[:, :, 1] = lbp1
            lbp[:, :, 2] = lbp2

        levels_histogram = np.array([])

        for i in range(levels):
            block_hist = histogram_by_block(lbp, 2**i, 2**i, bins = bins, dim = dim, mask = mask, radius = radius) 
            levels_histogram = np.concatenate((levels_histogram, block_hist))

        return levels_histogram

In [45]:
def get_all_hists(data, levels = 3, bins = 8, dim = "1", color = "RGB", masks = None, radius = 1, multiscale = False):
    full_dict = dict()
    
    for k, v in tqdm.tqdm(data.items()):
        if masks: 
            if masks[k].ndim == 3:
                full_dict[k] = get_full_hist_lbp(v, levels = levels, bins = bins, dim = dim, color = color, mask = masks[k][:, :, 0], multiscale = multiscale, radius = radius)
            else:
                full_dict[k] = get_full_hist_lbp(v, levels = levels, bins = bins, dim = dim, color = color, mask = masks[k], multiscale = multiscale, radius = radius)
        
        else:
            full_dict[k] = get_full_hist_lbp(v, levels = levels, bins = bins, dim = dim, color = color, mask = None, multiscale = multiscale, radius = radius)
        
    return full_dict

In [48]:
with open('./qsd1_w3/gt_corresps.pkl', 'rb') as f:
    cor1_w3 = pickle.load(f)
    
hists_qsd1_w3_radius_4_no_multiscale_64 = get_all_hists(qsd1_w3_median, levels = 3, bins = 64, dim = "1", color = "Gray", masks = tb_masks_inverted_qsd1_w3, radius = 4, multiscale = False)
hists_bbdd_radius_4_no_multiscale_64 = get_all_hists(bbdd, levels = 3, bins = 64, dim = "1", color = "Gray", masks = None, radius = 4, multiscale = False)

mAPK_qsd1_w3_lbp_k1 = calculateMAPK(hists_qsd1_w3_radius_4_no_multiscale_64, hists_bbdd_radius_4_no_multiscale_64, ground_truth = cor1_w3, topK=1)
mAPK_qsd1_w3_lbp_k5 = calculateMAPK(hists_qsd1_w3_radius_4_no_multiscale_64, hists_bbdd_radius_4_no_multiscale_64, ground_truth = cor1_w3, topK=5)
mAPK_qsd1_w3_lbp_k10 = calculateMAPK(hists_qsd1_w3_radius_4_no_multiscale_64, hists_bbdd_radius_4_no_multiscale_64, ground_truth = cor1_w3, topK=10)

print("Mean average precisions with different k values, using LBP descriptors:")
print(f"k=1 --> mAPK = {mAPK_qsd1_w3_lbp_k1}")
print(f"k=5 --> mAPK = {mAPK_qsd1_w3_lbp_k5}")
print(f"k=10 --> mAPK = {mAPK_qsd1_w3_lbp_k10}")

100%|█████████████████████████████████████████| 287/287 [01:30<00:00,  3.18it/s]
/var/folders/wp/ddc_g8m16t9_6p5h_jppb_xr0000gn/T/ipykernel_2157/3142807494.py:2: RuntimeWarning: invalid value encountered in divide
  result = ((h1 - h2)**2 / (h1 + h2))


Euclidean: 0.43333333333333335
X_Squared: 0.23333333333333334
L1: 0.3
Histogram_Intersection: 0.3
Hellinger_Kernel: 0.2
Euclidean: 0.48611111111111105
X_Squared: 0.29
L1: 0.3511111111111111
Histogram_Intersection: 0.3511111111111111
Hellinger_Kernel: 0.25555555555555554
Euclidean: 0.49087301587301585
X_Squared: 0.29
L1: 0.3548148148148148
Histogram_Intersection: 0.3548148148148148
Hellinger_Kernel: 0.26031746031746034
Mean average precisions with different k values, using LBP descriptors:
k=1 --> mAPK = 0.43333333333333335
k=5 --> mAPK = 0.48611111111111105
k=10 --> mAPK = 0.49087301587301585


## Discrete Cosine Transform (DCT)

1) Resize all images to 400x400
2) Apply DCT by Blocks
3) Flatten DCT results and concatenate
4) Normalize (?)
5) Compute Distances and mapK

In [49]:
def zigzag(matrix):
    rows, cols = len(matrix), len(matrix[0])
    result = [0] * (rows * cols)
    
    for i in range(rows):
        for j in range(cols):
            index = i * cols + j
            if (i + j) % 2 == 0:
                result[index] = matrix[i][j]
            else:
                result[index] = matrix[i][cols - 1 - j]
    
    return result

In [50]:
def dct_by_block(img, n_blocks_x, n_blocks_y, dim = "1"):
    
    M = math.ceil(img.shape[0]/n_blocks_x)
    N = math.ceil(img.shape[1]/n_blocks_y)
    tiles = [img[x:x+M, y:y+N] for x in range(0, img.shape[0], M) for y in range(0, img.shape[1], N)] 
    
    dcts = np.array([])
    
    for i in range(len(tiles)):
        
        if tiles[i].ndim == 2:
            # Grayscale
            dst = cv2.dct(np.float32(tiles[i]))
            tile_dct = np.array(zigzag(dst))
        elif tiles[i].ndim == 3 and tiles[i].shape[2] == 3:
            # BGR
            dst1 = cv2.dct(np.float32(tiles[i][:, :, 0]))
            dst2 = cv2.dct(np.float32(tiles[i][:, :, 1]))
            dst3 = cv2.dct(np.float32(tiles[i][:, :, 2]))
            tile_dct = np.concatenate([zigzag(dst1), zigzag(dst2), zigzag(dst3)]).flatten()
        else:
            raise Exception("Bad value of ndims: {}".format(tiles[i].ndim))
            
        dcts = np.concatenate((dcts, tile_dct))

    return dcts

        

def compute_dcts(qs, dim = "1", multiscale = False, levels = 3):
    dcts = {}
    
    for name, img in qs.items():
        # Resize images to 400x400
        img = cv2.resize(img, (400, 400))
        if dim == "1":
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if multiscale:
            levels_dct = np.array([])
    
            for i in range(levels):
                block_dct = dct_by_block(img, 2**i, 2**i, dim = dim)
                block_dct = (block_dct - np.mean(block_dct)) / np.std(block_dct)
                levels_dct = np.concatenate((levels_dct, block_dct))
                
            dcts[name] = levels_dct
        else:
            if img.ndim == 2:
                # Grayscale
                dst = cv2.dct(np.float32(img))
                dct = np.array(zigzag(dst))
            elif img.ndim == 3 and img.shape[2] == 3:
                # BGR
                dst1 = cv2.dct(np.float32(img[:, :, 0]))
                dst2 = cv2.dct(np.float32(img[:, :, 1]))
                dst3 = cv2.dct(np.float32(img[:, :, 2]))
                dct = np.concatenate([zigzag(dst1), zigzag(dst2), zigzag(dst3)]).flatten()
            else:
                raise Exception("Bad value of ndims: {}".format(tiles[i].ndim))

            dcts[name] = dct
            
    return dcts

---

## TASK 4

### Combining descriptors

### Copying the used methods from week 2 to computed color based descriptors

In [51]:
def get_2d1d_hist_w2(img, bins, mask = None):
    
    full_histogram = np.array([])
    
    if mask is not None: hist_1d = cv2.calcHist([img], [0], mask, [bins], [0, 256])
    else: hist_1d = cv2.calcHist([img], [0], None, [bins], [0, 256])
    if np.sum(hist_1d != 0): hist_1d /= np.sum(hist_1d)
    
    if mask is not None: hist_2d = cv2.calcHist([img, img], [1, 2], mask, [bins, bins], [0, 256, 0, 256])
    else: hist_2d = cv2.calcHist([img, img], [1, 2], None, [bins, bins], [0, 256, 0, 256])
    if np.sum(hist_2d != 0): hist_2d /= np.sum(hist_2d)
    
    hist = np.concatenate((hist_1d.flatten(), hist_2d.flatten()))

    return hist  

In [52]:
def histogram_by_block_w2(img, n_blocks_x, n_blocks_y, bins = 8, dim = "2d1d", mask = None):
    
    M = math.ceil(img.shape[0]/n_blocks_x)
    N = math.ceil(img.shape[1]/n_blocks_y)
    tiles = [img[x:x+M, y:y+N] for x in range(0, img.shape[0], M) for y in range(0, img.shape[1], N)]
    
    if mask is not None: 
        mask_tiles = [mask[x:x+M,y:y+N] for x in range(0, img.shape[0],M) for y in range(0, img.shape[1], N)]
       
    histogram = np.array([])
    
    for i in range(len(tiles)):
        if dim == "3":
            if mask is not None: tile_hist = cv2.calcHist([tiles[i], tiles[i], tiles[i]], [0, 1, 2], mask_tiles[i], [bins, bins, bins], [0, 256, 0, 256, 0, 256])
            else: tile_hist = cv2.calcHist([tiles[i]], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
        
        elif dim == "2d1d":           
            if mask is not None: tile_hist = get_2d1d_hist_w2(tiles[i], bins, mask = mask_tiles[i])
            else: tile_hist = get_2d1d_hist_w2(tiles[i], bins, mask = None)
        
        if np.sum(tile_hist) != 0:
            tile_hist /= np.sum(tile_hist)
            
        histogram = np.concatenate((histogram, tile_hist.flatten()))

    return histogram

In [53]:
def get_full_hist_w2(img, levels = 3, bins = 8, dim = "2d1d", color = "Lab", mask = None):
    
    if color == "Lab":
        image = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    else: 
        image = img.copy()
    
    levels_histogram = np.array([])
    
    for i in range(levels):
        block_hist = histogram_by_block_w2(image, 2**i, 2**i, bins = bins, dim = dim, mask = mask) 
        levels_histogram = np.concatenate((levels_histogram, block_hist))
        
    return levels_histogram

In [54]:
def get_all_hists_w2(data, levels = 3, bins = 8, dim = "2d1d", color = "Lab", masks = None):
    full_dict = dict()
    
    for k, v in data.items():
        if masks: 
            if masks[k].ndim == 3:
                full_dict[k] = get_full_hist_w2(v, levels = levels, bins = bins, dim = dim, color = color, mask = masks[k][:, :, 0])
            else:
                full_dict[k] = get_full_hist_w2(v, levels = levels, bins = bins, dim = dim, color = color, mask = masks[k])
        
        else:
            full_dict[k] = get_full_hist_w2(v, levels = levels, bins = bins, dim = dim, color = color, mask = None)
        
    return full_dict

In [55]:
def order_tls_and_brs_w2(tls_and_brs, centroids): 
    if len(tls_and_brs) != 2: return tls_and_brs

    if ((centroids[0][0] - centroids[1][0])**2 < (centroids[0][1] - centroids[1][1])**2): horizontal_ordering = False
    else: horizontal_ordering = True
        
    if horizontal_ordering:
        if centroids[0][0] < centroids[1][0]: return tls_and_brs
        else: return [tls_and_brs[1], tls_and_brs[0]] 
    else:
        if centroids[0][1] < centroids[1][1]: return tls_and_brs
        else: return [tls_and_brs[1], tls_and_brs[0]]

In [56]:
def divide_paintings_w2(img):
    
    h_i, w_i, c = img.shape
    
    top_limit = int(round(h_i/8))
    bottom_limit = int(h_i - top_limit)
    
    left_area = int(round(w_i/8))
    right_area = int(w_i - left_area)
    
    area_pixel= int(h_i*w_i)
    
    image = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    imageL = image[:, :, 0]
    imagea = image[:, :, 1]
    imageb = image[:, :, 2]
    
    #kernel = np.ones((2, 10),np.uint8)
    #thL = cv2.morphologyEx(imageL, cv2.MORPH_TOPHAT, kernel, iterations = 5)
    #bhL = cv2.morphologyEx(imageL, cv2.MORPH_BLACKHAT, kernel, iterations = 5)
    
    kernel = np.ones((2, 2),np.uint8)
    gradL = cv2.morphologyEx(imageL, cv2.MORPH_GRADIENT, kernel, iterations = 20)

    ret2, th2 = cv2.threshold(gradL,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    kernel = np.ones((round(h_i*0.001), round(w_i*0.001)),np.uint8)
    opening1 = cv2.morphologyEx(th2, cv2.MORPH_OPEN, kernel, iterations = 3)
    
    
    # -----------------------------------------------------------------------
    analysis = cv2.connectedComponentsWithStats(opening1, 4, cv2.CV_32S) 
    (totalLabels, label_ids, values, centroid) = analysis 

    # Initialize a new image to store  
    # all the output components 
    output1 = np.zeros(imageL.shape, dtype="uint8") 

    # Loop through each component 
    
    mask = np.zeros([h_i,w_i], dtype=np.uint8)
    cont=0
    x1_prev= -10000
    for i in range(1, totalLabels): 
        
          # Area of the component 
        area = values[i, cv2.CC_STAT_AREA]  
        
        (X, Y) = centroid[i]
         
        #if True:
        if (area > round(area_pixel*0.00001)):
            x1 = values[i, cv2.CC_STAT_LEFT] 
            y1 = values[i, cv2.CC_STAT_TOP] 
            w = values[i, cv2.CC_STAT_WIDTH] 
            h = values[i, cv2.CC_STAT_HEIGHT]
            
            pt1 = (x1, y1) 
            pt2 = (x1+ w, y1+ h)
            if (int(x1 - x1_prev) < int(round(w_i/4)) and (int(x1 - x1_prev) > int(round(-w_i/4)))):
                roi_corners = np.array([(x1_prev,y1_prev),(x1+w,y1), pt2,(x1_prev,y1+h)], dtype=np.int32)
            else:
                roi_corners = np.array([pt1,(x1+w,y1), pt2,(x1,y1+h)], dtype=np.int32)
        
               
            cv2.fillPoly(mask, [roi_corners], 255)
            output1 = cv2.bitwise_or(output1, mask)
            cont = cont+1
            x1_prev= x1
            y1_prev = y1                           
            
    if cont == 0:
        output1 = th2
        
    #print(cont)
    
    # ---------------------------------------------------------------
    
    kernel = np.ones((round(h_i*0.02), round(w_i*0.01)),np.uint8)
    opening2 = cv2.morphologyEx(output1, cv2.MORPH_OPEN, kernel, iterations = 15)
    
    # -----------------------------------------------------------------------
    analysis2 = cv2.connectedComponentsWithStats(opening2, 4, cv2.CV_32S) 
    (totalLabels2, label_ids2, values2, centroid2) = analysis2 

    # Initialize a new image to store  
    # all the output components 
    output2 = np.zeros(imageL.shape, dtype="uint8") 

    # Loop through each component 
    
    mask2 = np.zeros([h_i,w_i], dtype=np.uint8)
    cont=0
    
    tls_and_brs = []
    centroids = []
    
    for i in range(1, totalLabels2): 

        x1_prev= -10000
        
          # Area of the component 
        area = values2[i, cv2.CC_STAT_AREA]  
        
        (X, Y) = centroid2[i]
        
        if area > round(area_pixel*0.05):
        #if (area > round(area_pixel*0.2)) and ((Y<top_limit)or(Y>bottom_limit))and ((X>left_area)and(X<right_area)): 
            x1 = values2[i, cv2.CC_STAT_LEFT] 
            y1 = values2[i, cv2.CC_STAT_TOP] 
            w = values2[i, cv2.CC_STAT_WIDTH] 
            h = values2[i, cv2.CC_STAT_HEIGHT]
            
            pt1 = (x1, y1) 
            pt2 = (x1+ w, y1+ h)
            if (int(x1 - x1_prev) < int(round(w_i/4)) and (int(x1 - x1_prev) > int(round(-w_i/4)))):
                roi_corners2 = np.array([(x1_prev,y1_prev),(x1+w,y1), pt2,(x1_prev,y1+h)], dtype=np.int32)
            else:
                roi_corners2 = np.array([pt1,(x1+w,y1), pt2,(x1,y1+h)], dtype=np.int32)
        
               
            cv2.fillPoly(mask2, [roi_corners2], 255)
            output2 = cv2.bitwise_or(output2, mask2)
            cont = cont+1
            x1_prev= x1
            y1_prev = y1    
            
            tls_and_brs.append([roi_corners2[0][0], roi_corners2[0][1], roi_corners2[2][0], roi_corners2[2][1]])
            centroids.append([X, Y])
            
    if cont == 0:
        output2 = output1
    
    # ---------------------------------------------------------------
    '''
    # for visualization:
    fig, ax = plt.subplots(nrows=2, ncols=3, figsize = (10, 10))
    
    ax[0, 0].imshow(imageL, cmap = "gray")
    ax[0, 1].imshow(th2, cmap = "gray")
    ax[0, 2].imshow(opening1, cmap = "gray")
    
    ax[1, 0].imshow(output1, cmap = "gray")
    ax[1, 1].imshow(opening2, cmap = "gray")
    ax[1, 2].imshow(output2, cmap = "gray")
    '''
    
    tls_and_brs = order_tls_and_brs_w2(tls_and_brs, centroids)
    
    return tls_and_brs, output2

In [57]:
def task6_w2(qs, bbdd_hists, method = "L1", topK = 5, levels = 3, bins = 16, dim = "3", bins_1d = None, bins_2d = None, color = "Lab"):
    
    # depending on the method we want the largest or the closest score to select the topK
    if method in ["Euclidean", "L1", "X_squared"]: rev = False
    elif method in ["Histogram_Intersection", "Hellinger_Kernel"]: rev = True
    
    all_results = []
    
    for name, picture in qs.items():
        # we first have to divide the picture into paintings
        tls_and_brs = divide_paintings_w2(picture)[0]
            
        picture_results = []
        for painting_tl_and_br in tls_and_brs:
            # we are going to crop the picture to get each painting
            painting = picture[painting_tl_and_br[1] : painting_tl_and_br[3]+1, painting_tl_and_br[0] : painting_tl_and_br[2]+1]
            
            # we have to mask the text boxes
            not_rect_mask = get_name_boxs(painting)
            tl, br = get_tl_and_br(not_rect_mask)
            tr = (br[0], tl[1])
            bl = (tl[0], br[1])
            rect_mask = np.zeros((painting.shape[0], painting.shape[1]), dtype = np.uint8)
            roi_corners = np.array([tl, tr, br, bl], dtype = np.int32)
            cv2.fillPoly(rect_mask, [roi_corners], 255)
            
            # we have to invert the text box mask
            rect_mask = (255 - rect_mask)
            
            # we have to compute the histogram masking the text box
            hist = get_full_hist_w2(painting, levels = levels, bins = bins, dim = dim, color = color, mask = rect_mask)
            
            # compare with the bbdd histograms
            comparisons = compare(hist, bbdd_hists, method)
            
            # get the topK
            painting_topK = k_neighbours(comparisons, topK, rev = rev)
            
            # append topK to the picture results
            picture_results.append(painting_topK)
            
        # append picture results to all_results
        all_results.append(picture_results)
        
    return all_results

In [58]:
def get_painting_topK(img, hists_bbdd_RGB, tb_mask, dim="3", levels = 3, rev = True, color=None, bins = 16, method = "Histogram_Intersection", topK = 10):
    
    # we have to compute the histogram masking the text box
    hist = get_full_hist_w2(img, levels = levels, bins = bins, dim = dim, color = color, mask = tb_mask)

    # compare with the bbdd histograms
    comparisons = compare(hist, hists_bbdd_RGB, method)

    # get the topK
    img_topK = k_neighbours(comparisons, topK, rev = rev)
        
    return img_topK

In [59]:
def get_authors_and_paintings(authors, bbdd):
    
    full_dict = dict()
    
    for name, image in bbdd.items():
        
        author = authors[name].lower()
        
        if author == "": author = "unknown"
        
        if author in full_dict: author_dict = full_dict[author]
        else: author_dict = dict()
            
        author_dict[name] = image
        full_dict[author] = author_dict
         
    return full_dict  

In [60]:
def get_distance(img, tb_mask, method = "levenshtein"):
    
        
    tl, br = get_tl_and_br(tb_mask)
    tlx, tly = tl
    brx, bry = br
    margin = 0
    cropped_img = img[tly - margin : bry, tlx - margin : brx]
    ret2, th2 = cv2.threshold(cropped_img[:, :, 0], 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    extractedInformation_otsu = pytesseract.image_to_string(cropped_img).lower()
    otsu_split = extractedInformation_otsu.split('\n')
    min_score = 30
    closest_otsu = "unknown"
    for line in otsu_split:
        line_name, line_score = get_closest(authors, line, method)
        if line_score < min_score:
            closest_otsu = line_name
            min_score = line_score
            
    #print(extractedInformation_otsu, "     ", closest_otsu, "    ", min_score)
        
    return closest_otsu, min_score, cropped_img

In [61]:
def get_dct(img, dcts_bbdd, dim="3", multiscale = True, levels = 3, rev = False, method = "Euclidean", topK = 10):

    img = cv2.resize(img, (400, 400))
    if dim == "1":
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if multiscale:
        levels_dct = np.array([])

        for i in range(levels):
            block_dct = dct_by_block(img, 2**i, 2**i, dim = dim)
            block_dct = (block_dct - np.mean(block_dct)) / np.std(block_dct)
            levels_dct = np.concatenate((levels_dct, block_dct))
            
        dct = levels_dct
    else:
        if img.ndim == 2:
            # Grayscale
            dst = cv2.dct(np.float32(img))
            dct = np.array(zigzag(dst))
        elif img.ndim == 3 and img.shape[2] == 3:
            # BGR
            dst1 = cv2.dct(np.float32(img[:, :, 0]))
            dst2 = cv2.dct(np.float32(img[:, :, 1]))
            dst3 = cv2.dct(np.float32(img[:, :, 2]))
            dct = np.concatenate([zigzag(dst1), zigzag(dst2), zigzag(dst3)]).flatten()
        else:
            print("Error\n\n\n")
            raise Exception("Bad value of ndims: {}".format(tiles[i].ndim))

    # compare with the bbdd histograms
    comparisons = compare(dct, dcts_bbdd, method)

    # get the topK
    img_topK = k_neighbours(comparisons, topK)
        
    return img_topK

In [62]:
with open('./qsd2_w3/gt_corresps.pkl', 'rb') as f:
    cor2_w3 = pickle.load(f)

In [63]:
def get_lbp_topK(img, hists_lbp, tb_mask, dim="1", levels = 3, rev = True, color=None, bins = 16, method = "Histogram_Intersection", topK = 10, radius = 6, multiscale = False):
    
    # we have to compute the histogram masking the text box
    hist = get_full_hist_lbp(img, levels = levels, bins = bins, dim = dim, color = color, mask = tb_mask, radius = radius, multiscale = multiscale)

    # compare with the bbdd histograms
    comparisons = compare(hist, hists_lbp, method)

    # get the topK
    img_topK = k_neighbours(comparisons, topK, rev = rev)
        
    return img_topK

In [64]:
def spended_time(start, end):
    total_time = end - start
    return total_time

In [65]:
def combine_lists2(Color, DCT, LBP, color_W, dct_W, lbp_W):
    # Create a list to store the combined values for each ID
    combined_list = []

    # Create dictionaries to store the positions of IDs in each list
    color_positions = {id: i for i, id in enumerate(Color)}
    dct_positions = {id: i for i, id in enumerate(DCT)}
    lbp_positions = {id: i for i, id in enumerate(LBP)}

    # Iterate through the unique IDs
    for id in set(Color + DCT + LBP):
        color_weight = color_W * (1 / (color_positions[id] + 1)) if id in color_positions else 0
        dct_weight = dct_W * (1 / (dct_positions[id] + 1)) if id in dct_positions else 0
        lbp_weight = lbp_W * (1 / (lbp_positions[id] + 1)) if id in lbp_positions else 0

        combined_value = color_weight + dct_weight + lbp_weight
        combined_list.append((id, combined_value))

    # Sort the combined list by values in descending order
    combined_list.sort(key=lambda item: item[1], reverse=True)

    # Extract the IDs from the sorted list
    combined_top_10 = [id for id, _ in combined_list[:10]]

    return combined_top_10

In [84]:
def task4(qs, gt, method = "levenshtein"):
    
    start_time = timer()
    qs_medians = get_medians(qs)
    end_time = timer()
    print(f"Medians computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    bbdd_authors = get_dict_from_data(BBDD_PATH, extension=".txt")
    bbdd_author_paintings = get_authors_and_paintings(bbdd_authors, bbdd)
    end_time = timer()
    print(f"Authors and paintings dict computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    hists_bbdd = get_all_hists_w2(bbdd, levels = 3, bins = 32, dim = "2d1d", color = "Lab", masks = None)
    end_time = timer()
    print(f"Color histograms computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    dcts_bbdd = compute_dcts(bbdd, dim="3", multiscale = True)
    end_time = timer()
    print(f"DCTs computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    #lbps_bbdd = get_all_hists(bbdd, levels = 3, bins = 64, dim = "1", color = "Gray", masks = None, radius = 4, multiscale = False)
    lbps_bbdd = hists_bbdd_radius_4_no_multiscale_64
    end_time = timer()
    print(f"LBPs computed. Spended {spended_time(start_time, end_time):.3f} seconds")
    
    
    all_results_color = []
    all_results_dct = []
    all_results_lbp = []
    
    all_results = []
    
    for pic_name, picture in tqdm.tqdm(qs_medians.items()):
        print(pic_name, ":")
        # we first have to divide the picture into paintings
        tls_and_brs = divide_paintings_w2(picture)[0]
            
        picture_results_color = []
        picture_results_dct = []
        picture_results_lbp = []
        
        picture_results = []
        
        for painting_tl_and_br in tls_and_brs:
            # we are going to crop the picture to get each painting
            median_painting = picture[painting_tl_and_br[1] : painting_tl_and_br[3], painting_tl_and_br[0] : painting_tl_and_br[2]]
            normal_painting = qs[pic_name][painting_tl_and_br[1] : painting_tl_and_br[3], painting_tl_and_br[0] : painting_tl_and_br[2]]
            
            print(f"Median painting shape: {median_painting.shape}, Normal painting shape: {normal_painting.shape}")
            
            #------------------------------------
            # MEDIAN
            # we have to mask the text boxes
            median_not_rect_mask = get_name_boxs(median_painting)
            tl, br = get_tl_and_br(median_not_rect_mask)
            tr = (br[0], tl[1])
            bl = (tl[0], br[1])
            median_rect_mask = np.zeros((median_painting.shape[0], median_painting.shape[1]), dtype = np.uint8)
            median_roi_corners = np.array([tl, tr, br, bl], dtype = np.int32)
            cv2.fillPoly(median_rect_mask, [median_roi_corners], 255)
            
            median_name, median_score, median_cropped_img = get_distance(median_painting, median_rect_mask, method = method)
            #------------------------------------
            
            
            #------------------------------------
            # NORMAL
            # we have to mask the text boxes
            normal_not_rect_mask = get_name_boxs(normal_painting)
            tl, br = get_tl_and_br(normal_not_rect_mask)
            tr = (br[0], tl[1])
            bl = (tl[0], br[1])
            normal_rect_mask = np.zeros((normal_painting.shape[0], normal_painting.shape[1]), dtype = np.uint8)
            normal_roi_corners = np.array([tl, tr, br, bl], dtype = np.int32)
            cv2.fillPoly(normal_rect_mask, [normal_roi_corners], 255)
            
            normal_name, normal_score, normal_cropped_img = get_distance(normal_painting, normal_rect_mask, method = method)
            #------------------------------------
            
            if normal_score < median_score:
                score = normal_score
                name = normal_name
                cropped_img = normal_cropped_img
                pic = normal_painting
                tb_mask = normal_rect_mask
            else: 
                score = median_score
                name = median_name
                cropped_img = median_cropped_img
                pic = median_painting
                tb_mask = median_rect_mask
                
            inverted_tb_mask = 255 - tb_mask
            with open("./QST2/method1/"+pic_name+".txt", 'a') as file:
                file.write(string.capwords(name, sep = None)+"\n")
            
            if score <= 4:
                searched_dict = bbdd_author_paintings[name]
                used_RGB = dict()
                used_DCT = dict()
                used_LBP = dict()
                for key in searched_dict.keys():
                    used_RGB[key] = hists_bbdd[key]
                    used_DCT[key] = dcts_bbdd[key]
                    used_LBP[key] = lbps_bbdd[key]
                    
            else:
                used_RGB = hists_bbdd
                used_DCT = dcts_bbdd
                used_LBP = lbps_bbdd
                    
                    
            # COLOR
            color_topk = get_painting_topK(pic, used_RGB, dim = "2d1d", color="Lab", bins = 32, method = "Histogram_Intersection", topK = 10, tb_mask = inverted_tb_mask)
                 
            # DCT  
            dct_topk = get_dct(pic, used_DCT, dim="3", multiscale = True, levels = 3, rev = False, method = "Euclidean", topK = 10)
            
            # LBP
            lbp_topk = get_lbp_topK(pic, used_LBP, inverted_tb_mask, dim="1", levels = 3, rev = True, color="Gray", bins = 64, method = "Histogram_Intersection", topK = 10, radius = 4, multiscale = False)
            
            print(f"Color top 10: {color_topk}\nDCT top 10: {dct_topk}\nLBP top 10: {lbp_topk}")

            out = combine_lists2(color_topk, dct_topk, lbp_topk, color_W=0.5, dct_W=0.3, lbp_W=0.2)
            while len(out) < 10:
                out.append(0)
            print(f"Overall top 10: {out}")
                
            while len(color_topk) < 10: color_topk.append(0)
            while len(dct_topk) < 10: dct_topk.append(0)
            while len(lbp_topk) < 10: lbp_topk.append(0)
        
            picture_results_color.append(color_topk)
            picture_results_dct.append(dct_topk)
            picture_results_lbp.append(lbp_topk)
            
            picture_results.append(out)

            
        print("---------")
        print("---------\n\n")
            
        all_results_color.append(picture_results_color)
        all_results_dct.append(picture_results_dct)
        all_results_lbp.append(picture_results_lbp)
        
        all_results.append(picture_results)
        
    return all_results_color, all_results_dct, all_results_lbp, all_results
        

In [67]:
results_task4 = task4(qsd2_w3, cor2_w3)

Medians computed. Spended 0.166 seconds
Authors and paintings dict computed. Spended 0.047 seconds
Color histograms computed. Spended 6.595 seconds
DCTs computed. Spended 83.437 seconds
LBPs computed. Spended 0.001 seconds


  0%|                                                    | 0/30 [00:00<?, ?it/s]

00000 :
Median painting shape: (416, 467, 3), Normal painting shape: (416, 467, 3)


  3%|█▍                                          | 1/30 [00:01<00:36,  1.25s/it]

Color top 10: [282, 32, 246, 109, 210]
DCT top 10: [109, 246, 32, 210, 282]
LBP top 10: [109, 32, 246, 282, 210]
Overall top 10: [109, 282, 32, 246, 210, 0, 0, 0, 0, 0]
---------
---------


00001 :
Median painting shape: (1753, 1528, 3), Normal painting shape: (1753, 1528, 3)
Color top 10: [116, 248, 60, 40, 168, 81, 261, 197, 0, 234]
DCT top 10: [78, 36, 6, 65, 164, 108, 209, 53, 283, 44]
LBP top 10: [163, 145, 3, 162, 161, 58, 57, 119, 170, 201]
Overall top 10: [116, 78, 248, 163, 60, 36, 40, 145, 168, 6]
Median painting shape: (800, 865, 3), Normal painting shape: (800, 865, 3)


...

Color top 10: [76, 103, 140, 43, 225, 113, 163, 249, 192, 88]
DCT top 10: [164, 94, 74, 97, 109, 53, 184, 18, 1, 194]
LBP top 10: [201, 161, 162, 112, 130, 145, 119, 170, 137, 163]
Overall top 10: [76, 164, 103, 201, 140, 94, 43, 161, 225, 74]
---------
---------


00028 :
Median painting shape: (553, 409, 3), Normal painting shape: (553, 409, 3)


 97%|█████████████████████████████████████████▌ | 29/30 [03:13<00:02,  2.31s/it]

Color top 10: [183, 67, 266]
DCT top 10: [183, 67, 266]
LBP top 10: [183, 266, 67]
Overall top 10: [183, 67, 266, 0, 0, 0, 0, 0, 0, 0]
---------
---------


00029 :
Median painting shape: (1825, 1766, 3), Normal painting shape: (1825, 1766, 3)


100%|███████████████████████████████████████████| 30/30 [03:23<00:00,  6.77s/it]

Color top 10: [137, 51, 163, 225, 248, 104, 168, 58, 81, 154]
DCT top 10: [228, 51, 58, 104, 215, 225, 163, 137, 192, 82]
LBP top 10: [225, 192, 168, 259, 170, 228, 58, 215, 113, 130]
Overall top 10: [137, 51, 225, 228, 163, 58, 104, 168, 192, 248]
---------
---------




In [74]:
#Color
compute_mapk(cor2_w3,results_task4[0],5)

0.6923076923076923

In [69]:
#DCT
compute_mapk(cor2_w3,results_task4[1],5)

0.5884615384615385

In [70]:
#LBP
compute_mapk(cor2_w3,results_task4[2],5)

0.3803418803418803

In [75]:
#Combine
compute_mapk(cor2_w3,results_task4[3],1)

0.6666666666666666

In [85]:
def task4_test1(qs, method = "levenshtein"):
    
    start_time = timer()
    qs_medians = get_medians(qs)
    end_time = timer()
    print(f"Medians computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    bbdd_authors = get_dict_from_data(BBDD_PATH, extension=".txt")
    bbdd_author_paintings = get_authors_and_paintings(bbdd_authors, bbdd)
    end_time = timer()
    print(f"Authors and paintings dict computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    hists_bbdd = get_all_hists_w2(bbdd, levels = 3, bins = 32, dim = "2d1d", color = "Lab", masks = None)
    end_time = timer()
    print(f"Color histograms computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    dcts_bbdd = compute_dcts(bbdd, dim="3", multiscale = True)
    end_time = timer()
    print(f"DCTs computed. Spended {spended_time(start_time, end_time):.3f} seconds")

    start_time = timer()
    lbps_bbdd = get_all_hists(bbdd, levels = 3, bins = 64, dim = "1", color = "Gray", masks = None, radius = 5, multiscale = False)
    end_time = timer()
    print(f"LBPs computed. Spended {spended_time(start_time, end_time):.3f} seconds")
    
    
    all_results_color = []
    all_results_dct = []
    all_results_lbp = []
    all_results_text=[]
    
    all_results = []
    
    for pic_name, picture in tqdm.tqdm(qs_medians.items()):
        print(pic_name, ":")

            
        picture_results_color = []
        picture_results_dct = []
        picture_results_lbp = []
        
        picture_results = []
        
        
        #------------------------------------
        # MEDIAN
        # we have to mask the text boxes
        median_painting=picture
        median_not_rect_mask = get_name_boxs(median_painting)
        tl, br = get_tl_and_br(median_not_rect_mask)
        tr = (br[0], tl[1])
        bl = (tl[0], br[1])
        median_rect_mask = np.zeros((median_painting.shape[0], median_painting.shape[1]), dtype = np.uint8)
        median_roi_corners = np.array([tl, tr, br, bl], dtype = np.int32)
        cv2.fillPoly(median_rect_mask, [median_roi_corners], 255)

        median_name, median_score, median_cropped_img = get_distance(median_painting, median_rect_mask, method = method)
        #------------------------------------


        #------------------------------------
        # NORMAL
        # we have to mask the text boxes
        normal_painting = qs[pic_name]
        normal_not_rect_mask = get_name_boxs(normal_painting)
        tl, br = get_tl_and_br(normal_not_rect_mask)
        tr = (br[0], tl[1])
        bl = (tl[0], br[1])
        normal_rect_mask = np.zeros((normal_painting.shape[0], normal_painting.shape[1]), dtype = np.uint8)
        normal_roi_corners = np.array([tl, tr, br, bl], dtype = np.int32)
        cv2.fillPoly(normal_rect_mask, [normal_roi_corners], 255)

        normal_name, normal_score, normal_cropped_img = get_distance(normal_painting, normal_rect_mask, method = method)
        #------------------------------------

        if normal_score < median_score:
            score = normal_score
            name = normal_name
            cropped_img = normal_cropped_img
            pic = normal_painting
            tb_mask = normal_rect_mask
        else: 
            score = median_score
            name = median_name
            cropped_img = median_cropped_img
            pic = median_painting
            tb_mask = median_rect_mask

        inverted_tb_mask = 255 - tb_mask
        with open("./QST1/method1/"+pic_name+".txt", 'w') as file:
            file.write(string.capwords(name, sep = None)+"\n")
            
        text_topk = [int(key) for key in bbdd_author_paintings[name].keys()]
        if len(text_topk)>10: text_topk=text_topk[:10]
        while len(text_topk)<10: text_topk.append(0)
        all_results_text.append(text_topk)
        
        if score <= 4:
            searched_dict = bbdd_author_paintings[name]
            used_RGB = dict()
            used_DCT = dict()
            used_LBP = dict()
            for key in searched_dict.keys():
                used_RGB[key] = hists_bbdd[key]
                used_DCT[key] = dcts_bbdd[key]
                used_LBP[key] = lbps_bbdd[key]

        else:
            used_RGB = hists_bbdd
            used_DCT = dcts_bbdd
            used_LBP = lbps_bbdd


        # COLOR
        color_topk = get_painting_topK(pic,used_RGB, dim = "2d1d", color="Lab", bins = 32, method = "Histogram_Intersection", topK = 10, tb_mask = inverted_tb_mask)

        # DCT  
        dct_topk = get_dct(pic, used_DCT, dim="3", multiscale = True, levels = 3, rev = False, method = "Euclidean", topK = 10)

        # LBP
        lbp_topk = get_lbp_topK(pic, used_LBP, inverted_tb_mask, dim="1", levels = 3, rev = True, color="Gray", bins = 64, method = "Histogram_Intersection", topK = 10, radius = 5, multiscale = False)

        print(f"Color top 10: {color_topk}\nDCT top 10: {dct_topk}\nLBP top 10: {lbp_topk}\ntext top 10: {text_topk}")

        out = combine_lists2(color_topk, dct_topk, lbp_topk, color_W=0.5, dct_W=0.3, lbp_W=0.2)
        while len(out) < 10:
            out.append(0)
        print(f"Overall top 10: {out}")

        while len(color_topk) < 10: color_topk.append(0)
        while len(dct_topk) < 10: dct_topk.append(0)
        while len(lbp_topk) < 10: lbp_topk.append(0)

        picture_results_color.append(color_topk)
        picture_results_dct.append(dct_topk)
        picture_results_lbp.append(lbp_topk)

        picture_results.append(out)

            
        print("---------")
        print("---------\n\n")
            
        all_results_color.append(color_topk)
        all_results_dct.append(dct_topk)
        all_results_lbp.append(lbp_topk)
        
        all_results.append(out)
        
    return all_results_color, all_results_dct, all_results_lbp, all_results, all_results_text

### TESTS

In [87]:
QST1_W3_PATH = './qst1_w3/'
QST2_W3_PATH = './qst2_w3/'

qst1_w3 = get_dict_from_data(QST1_W3_PATH)
qst2_w3 = get_dict_from_data(QST2_W3_PATH)

resultados_qst2 = task4(qst2_w3,None)
resultados_qst1 = task4_test1(qst1_w3)

Medians computed. Spended 0.061 seconds
Authors and paintings dict computed. Spended 0.039 seconds
Color histograms computed. Spended 7.506 seconds
DCTs computed. Spended 83.157 seconds
LBPs computed. Spended 0.000 seconds


  0%|                                                    | 0/30 [00:00<?, ?it/s]

00000 :
Median painting shape: (1729, 1527, 3), Normal painting shape: (1729, 1527, 3)


  3%|█▍                                          | 1/30 [00:13<06:40, 13.81s/it]

Color top 10: [248, 81, 197, 21, 60, 116, 156, 99, 94, 140]
DCT top 10: [78, 36, 6, 65, 108, 164, 116, 283, 209, 53]
LBP top 10: [190, 116, 113, 215, 37, 40, 119, 271, 244, 63]
Overall top 10: [248, 78, 81, 116, 190, 197, 36, 21, 60, 6]
---------
---------


00001 :
Median painting shape: (612, 781, 3), Normal painting shape: (612, 781, 3)


...

Color top 10: [258, 35]
DCT top 10: [258, 35]
LBP top 10: [258, 35]
text top 10: [35, 258, 0, 0, 0, 0, 0, 0, 0, 0]
Overall top 10: [258, 35, 0, 0, 0, 0, 0, 0, 0, 0]
---------
---------


00048 :


 98%|██████████████████████████████████████████▏| 49/50 [02:52<00:02,  2.76s/it]

Color top 10: [35, 258]
DCT top 10: [35, 258]
LBP top 10: [35, 258]
text top 10: [35, 258, 0, 0, 0, 0, 0, 0, 0, 0]
Overall top 10: [35, 258, 0, 0, 0, 0, 0, 0, 0, 0]
---------
---------


00049 :


100%|███████████████████████████████████████████| 50/50 [02:54<00:00,  3.50s/it]

Color top 10: [147, 58, 78, 248, 56, 35, 23, 237, 278, 234]
DCT top 10: [6, 78, 164, 36, 108, 65, 209, 53, 106, 147]
LBP top 10: [225, 57, 168, 259, 218, 192, 140, 223, 103, 201]
text top 10: [75, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Overall top 10: [147, 78, 6, 58, 225, 248, 56, 57, 164, 35]
---------
---------




In [79]:
with open('./QST2/result.pkl', 'wb') as f:
    pickle.dump(resultados_qst2[3], f)

with open('./QST1/result.pkl', 'wb') as f:
    pickle.dump(resultados_qst1[3], f)

In [82]:
with open('./QST2/result.pkl', 'rb') as f:
    results = pickle.load(f)
    


In [83]:
results

[[[248, 78, 81, 116, 190, 197, 36, 21, 60, 6]],
 [[53, 55, 124, 198, 0, 0, 0, 0, 0, 0]],
 [[225, 248, 51, 58, 82, 215, 163, 130, 137, 168]],
 [[211, 149, 203, 251, 0, 0, 0, 0, 0, 0]],
 [[132, 130, 57, 88, 103, 108, 140, 31, 184, 113],
  [31, 106, 57, 47, 286, 164, 174, 88, 236, 108],
  [271, 135, 69, 234, 232, 84, 132, 63, 237, 275]],
 [[28, 220, 41, 172, 0, 0, 0, 0, 0, 0], [41, 220, 28, 172, 0, 0, 0, 0, 0, 0]],
 [[251, 149, 203, 211, 0, 0, 0, 0, 0, 0]],
 [[25, 242, 8, 0, 0, 0, 0, 0, 0, 0]],
 [[286, 119, 260, 23, 0, 0, 0, 0, 0, 0]],
 [[260, 286, 119, 84, 36, 248, 21, 161, 164, 88]],
 [[170, 228, 225, 215, 154, 160, 51, 259, 130, 248]],
 [[215, 239, 192, 51, 225, 39, 163, 58, 228, 113]],
 [[183, 67, 266, 0, 0, 0, 0, 0, 0, 0]],
 [[76, 249, 43, 136, 0, 0, 0, 0, 0, 0]],
 [[73, 159, 108, 189, 278, 97, 36, 279, 17, 126],
  [27, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[160, 110, 140, 133, 192, 58, 57, 259, 218, 244]],
 [[164, 74, 46, 107, 0, 0, 0, 0, 0, 0]],
 [[32, 109, 246, 210, 282, 0, 0, 0, 0, 0],
